In [1]:
import os
import sys
from pathlib import Path

base_dir=f'{Path(os.getcwd()).parent}'
sys.path.append(base_dir)


In [2]:
import soundfile as sf
import librosa
import numpy as np

def readAudioFile(audio_path):
    audio, sample_rate = sf.read(audio_path)
    if audio.ndim > 1:
        audio = audio[:, 0]
    if sample_rate != 16000:
        audio = librosa.resample(audio, orig_sr=sample_rate, target_sr=16000)
    return audio.astype(np.float32)
readAudioFile("/mnt/e/dataset/jsut_ver1.1/loanword128/wav/LOANWORD128_064.wav")

array([0.00355952, 0.00581642, 0.00446812, ..., 0.00274543, 0.00263134,
       0.00282599], dtype=float32)

In [3]:
import numpy as np

whole_audio = np.concatenate([readAudioFile("/mnt/e/dataset/jsut_ver1.1/loanword128/wav/LOANWORD128_064.wav"),
                              readAudioFile("/mnt/e/dataset/jsut_ver1.1/loanword128/wav/LOANWORD128_065.wav"),
                              readAudioFile("/mnt/e/dataset/jsut_ver1.1/loanword128/wav/LOANWORD128_066.wav"),
                              ]).astype(np.float32)

import IPython.display as ipd
ipd.Audio(whole_audio, rate=16000)

In [4]:
from faster_whisper import WhisperModel

model: WhisperModel =WhisperModel(
    "medium",
    device="cuda",
    compute_type="float32",
    download_root=f"{base_dir}/.model_cache",
)


/home/sylx/project/altbot/ai-service/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
segments,info = model.transcribe(whole_audio,
                                    condition_on_previous_text=False,
                                    prefix="",initial_prompt="",
                                    compression_ratio_threshold=1.5,
                                    beam_size=1,language="ja",vad_filter=True,
                                    temperature=[0.0],
                                    word_timestamps=True)
segements=list(segments)
print([s.text for s in segements])


['クエゼリン島の戦いとは、日本軍の守るクエゼリン艦将へアメリカ軍が侵攻して行われた戦闘である。', 'クオンサウンは韓国の俳優である。', 'ガルネリはイタリア出身の弦楽器製作者一族、または彼らが製作した弦楽器である。']


In [6]:
# 分割して認識させる
window_size=int(16000*2)

for i in range(0,len(whole_audio),window_size):
    audio = whole_audio[i:i+window_size]
    segments,info = model.transcribe(audio,
                                    condition_on_previous_text=False,
                                    prefix="",initial_prompt="",
                                    compression_ratio_threshold=1.5,
                                    beam_size=1,language="ja",vad_filter=True,
                                    temperature=[0.0],
                                    word_timestamps=True)
    segements=list(segments)
    print(f"{i/16000:0.2f}: {[s.text for s in segements]}")
ipd.Audio(whole_audio[0:window_size], rate=16000)

0.00: ['ケゼリン島の戦いとは']
2.00: ['日本軍の守るクエゼリン官商へ']
4.00: ['アメリカ軍が進行して行われた']
6.00: ['戦闘である']
8.00: ['純は韓国の俳優である。']
10.00: ['ガルネリはイタリア出身です。']
12.00: ['現学期制作者一族']
14.00: ['または彼らが生息した']
16.00: ['弦楽器である']


In [12]:
# stride分を重複させる
stride=int(16000*1)
window_size=int(16000*2)

audio_chunks=[]
prompt=""
for i in range(0,len(whole_audio),window_size):
    if i >= stride:
        audio = np.concatenate([whole_audio[i-stride:i],whole_audio[i:i+window_size]])
    else:
        audio = whole_audio[i:i+window_size]
    audio_chunks.append(audio)
    segments,info = model.transcribe(audio,
                                    condition_on_previous_text=False,
                                    prefix="",initial_prompt=prompt,
                                    compression_ratio_threshold=1.5,
                                    beam_size=3,language="ja",vad_filter=True,best_of=2,
                                    temperature=[0.0,0.2,0.4],
                                    word_timestamps=True)
    segements=list(segments)
    whole_text="".join([s.text for s in segements])
    print(f"{i/16000:0.2f}: {whole_text}")
    #prompt=whole_text

0.00: ケゼリン島の戦いとは
2.00: この戦いとは、日本軍の守るクエゼリン艦将へ。
4.00: クエゼリン干渉へアメリカ軍が侵攻して行われた
6.00: 人工して行われた戦闘である。コンサンは
8.00: コンサウンは韓国の俳優である。
10.00: ガルネリはイタリア出身です。
12.00: イタリア出身の弦楽規制作者一族
14.00: 作者一族、または彼らが生息した 家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族の家族
16.00: これらが生息した弦楽器である。
